In [ ]:
#r "nuget: MathNet.Numerics.FSharp, 4.15.0"

Installed package MathNet.Numerics.FSharp version 4.15.0

In [ ]:
open MathNet.Numerics.LinearAlgebra

In [ ]:
let a = fun x -> cos x
// b = 0, t = 0

// u = x ^ 3 + t ^ 3
let f = fun x t -> 3. * t ** 2. - 6. * x * cos x
let phi = fun (x: float) -> x ** 3.
let alpha = fun (t: float) -> t ** 3.
let beta = fun (t: float) -> 3.

let left = 0.
let right = 1.
let T = 0.1

In [ ]:
let n = 5
let h = (right - left) / float n
let x = [ left .. h .. right ]

let m = 5
let tau = T / float m
let t = [ 0. .. tau .. T ]

In [ ]:
let explicitScheme n m = 
    let u = DenseMatrix.zero<float> (n + 1) (m + 1)

    x
    |> List.iteri (fun i x -> u.[i, 0] <- phi x)

    let l i k = a x.[i] * (u.[i + 1, k] - 2. * u.[i, k] + u.[i - 1, k]) / h ** 2.

    for k = 1 to m do
        for i = 1 to n - 1 do
            u.[i, k] <- u.[i, k - 1] + tau * (l i (k - 1) + f x.[i] t.[k - 1])
        u.[0, k] <- alpha t.[k]
        u.[n, k] <- 2. / 3. * h * beta t.[k] + 4. * u.[n - 1, k] - u.[n - 2, k]

    u

In [ ]:
let weightScheme n m = 
    let u = DenseMatrix.zero<float> (n + 1) (m + 1)

    x
    |> List.iteri (fun i x -> u.[i, 0] <- phi x)

    let l i k = a x.[i] * (u.[i + 1, k] - 2. * u.[i, k] + u.[i - 1, k]) / h ** 2.

    for k = 1 to m do
        let m = DenseMatrix.zero<float> (n + 1) (n + 1)

        m.[0, 0] <- 1.
        m.[0, 1] <- 0.

        m.[n, n - 1] <- -1. / h
        m.[n, n] <- -1. / h

        let g = DenseVector.zero<float> (n + 1)
        g.[0] <- alpha t.[k]
        g.[n] <- beta t.[k]

        for i = 1 to n - 1 do 
            m.[i, i - 1] <- a x.[i] / h ** 2.
            m.[i, i] <- -2. * a x.[i] / h ** 2.
            m.[i, i + 1] <- a x.[i] / h ** 2.
            
            g.[i] <- -1. / tau * u.[i, k - 1] - (1. - 1. / 2.) * l i (k - 1) - f x.[i] t.[k]

        let s = m.Solve g

        for i = 0 to n do
            u.[i, k] <- s.[i]

    u

In [ ]:
(Matrix.toRowSeq <| explicitScheme n m)
|> Seq.map (fun row -> {| A = row.[0]; B = row.[1]; C = row.[2]; D = row.[3]; E = row.[4] |})

index,A,B,C,D,E
0,0,-0.009866666666666682,-0.016864888888888863,-0.018869629629629556,-0.022121991543210068
1,0.008000000000000002,0.008800000000000002,0.003757333333333358,0.0015244444444444606,-0.0009078518518519253
2,0.06400000000000002,0.06480000000000005,0.06562400000000003,0.06270666666666651,0.0627345672222225
3,0.2160000000000001,0.21680000000000008,0.21762399999999976,0.2178804600000003,0.21631165599999927
4,0.5120000000000001,0.5127999999999998,0.5128716000000002,0.5136399199999996,0.5135076550000005
5,1,1.000008,1.000064,1.000216,1.000512


In [ ]:
DenseMatrix.init (n + 1) (m + 1) (fun i k -> x.[i] ** 3. + t.[k] ** 3.)
|> Matrix.toRowSeq
|> Seq.map (fun row -> {| A = row.[0]; B = row.[1]; C = row.[2]; D = row.[3]; E = row.[4] |})

index,A,B,C,D,E
0,0,8.000000000000001E-06,6.400000000000001E-05,0.00021599999999999996,0.0005120000000000001
1,0.008000000000000002,0.008008000000000001,0.008064000000000002,0.008216000000000001,0.008512000000000002
2,0.06400000000000002,0.06400800000000001,0.06406400000000001,0.06421600000000001,0.06451200000000001
3,0.2160000000000001,0.21600800000000012,0.21606400000000012,0.2162160000000001,0.21651200000000012
4,0.5120000000000001,0.5120080000000001,0.5120640000000001,0.5122160000000001,0.5125120000000001
5,1,1.000008,1.000064,1.000216,1.000512


7 вар

In [ ]:
let p = fun x -> 1. + x
// b = 0, t = 0

// u = x ^ 3 + t ^ 3
let f = fun x t -> 3. * (t ** 2. - 3. * x ** 2. - 2. * x)
let phi = fun (x: float) -> x ** 3.
let alpha = fun (t: float) -> 0.
let beta = fun (t: float) -> t ** 3. + 1.

let left = 0.
let right = 1.
let T = 0.1

In [ ]:
let n = 5
let h = (right - left) / float n
let x = [ left .. h .. right ]

let m = 5
let tau = T / float m
let t = [ 0. .. tau .. T ]

In [ ]:
let explicitScheme n m = 
    let u = DenseMatrix.zero<float> (n + 1) (m + 1)

    x
    |> List.iteri (fun i x -> u.[i, 0] <- phi x)

    let l i k = p ((float i + 0.5) / float n) * (u.[i + 1, k] - u.[i, k]) / h / h - p ((float i - 0.5) / float n) * (u.[i, k] - u.[i - 1, k]) / h / h

    for k = 1 to m do
        for i = 1 to n - 1 do
            u.[i, k] <- u.[i, k - 1] + tau * (l i (k - 1) + f x.[i] t.[k - 1])
        u.[0, k] <- (-2. * alpha t.[k] / float n + 4. * u.[1, k] - u.[2, k]) / 3.
        u.[n, k] <- beta t.[k]

    u

In [ ]:
let weightScheme w n m = 
    let u = DenseMatrix.zero<float> (n + 1) (m + 1)

    x
    |> List.iteri (fun i x -> u.[i, 0] <- phi x)

    let l i k = p ((float i + 0.5) / float n) * (u.[i + 1, k] - u.[i, k]) / h / h - p ((float i - 0.5) / float n) * (u.[i, k] - u.[i - 1, k]) / h / h

    let a = [ for i = 0 to n - 2 do yield (w * p ((float i + 0.5) / float n) / h / h) ] @ [0.]
    let b = [ yield float n; for i = 1 to n - 1 do yield (w * ( p ((float i + 0.5) / float n) + p ((float i - 0.5) / float n)) / h / h + 1. / tau) ] @ [-1.]
    let c = [ yield float n; for i = 1 to n - 1 do yield (w * p ((float i + 0.5) / float n) / h ** 2.) ]

    let slv (a: float list) (b: float list) (c: float list) (g: float list) = 
        let n = List.length a

        let mutable s = [ c.[0] / b.[0] ]
        let mutable t = [ -g.[0] / b.[0] ]

        for i = 1 to n - 1 do
            let d = b.[i] - a.[i - 1] * s.[i - 1]
            s <- List.append s [ c.[i] / d ]
            t <- List.append t [ (a.[i - 1] * t.[i - 1] - g.[i]) / d]

        let mutable y = [| (a.[n - 1] * t.[n - 1] - g.[n]) / (b.[n] - a.[n - 1] * s.[n - 1]) |]

        for i = n - 1 downto 0 do
            y <- Array.append y [| s.[i] * y.[^0] + t.[i] |]

        printfn "%i" <| Array.length y

        for i = 0 to Array.length y / 2 - 1 do
            let temp = y.[i]
            y.[i] <- y.[^i]   
            y.[^i] <- temp

        y

    for k = 1 to m do
        let g = DenseVector.init (n + 1) (fun i ->
            if i = 0 then alpha t.[k]
            elif i = n then beta t.[k]
            else -1. / tau * u.[i, k - 1] - (1. - w) * l i (k - 1) - f x.[i] (t.[k] - (1. - w) * tau)
        ) 

        let s = slv a b c (Vector.toList g)

        for i = 0 to n do
            u.[i, k] <- s.[i]

    u

In [ ]:
(Matrix.toRowSeq <| weightScheme 0. n m)
|> Seq.map (fun row -> {| A = row.[0]; B = row.[1]; C = row.[2]; D = row.[3]; E = row.[4] |})

1

4

3

2

1

...

2

1

0

6

index,A,B,C,D,E
0,0,0.008799999999999999,0.014024000000000028,0.016460000000000044,0.019873999999999833
1,0.008000000000000002,0.008799999999999999,0.014024000000000028,0.016460000000000044,0.019873999999999833
2,0.06400000000000002,0.06480000000000005,0.06562400000000007,0.06937999999999973,0.06977334500000038
3,0.2160000000000001,0.21680000000000013,0.21762399999999968,0.21788046000000036,0.22131665599999917
4,0.5120000000000001,0.5127999999999997,0.5128716000000002,0.5136399199999996,0.5135076550000006
5,1,1.000008,1.000064,1.000216,1.000512


In [ ]:
[|1;2;3|].[^0]

3